## Game Theory Task 1

### Simultaneous QUATRO-UNO

###      
     Team Members:
     Ali Shaker - 20201701725      Diaa Amr - 20201701720        Rawan Mohsen - 20201701717      Mariam Alaa - 20201701739

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [126]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter.ttk import Combobox
from PIL import ImageTk, Image
from tkinter.ttk import *
import os
import time
import json

In [127]:
master = tk.Tk()
master.title("Simultaneous QUATRO-UNO")
master.geometry('1000x450')
master.resizable(False,False)

''

In [128]:
title_var = tk.StringVar()
title = tk.Label(master, textvariable=title_var, font=('Comic Sans MS', 20), bg='LightCyan2')
title_var.set("Simultaneous QUATRO-UNO")
title.place(x=295,y=27)

player1_Var = tk.StringVar()
player1 = tk.Label(master, textvariable=player1_Var, font=('Comic Sans MS', 14), bg='LightCyan2')
player1_Var.set("Player 1")
player1.place(x=200,y=120)

player2_Var = tk.StringVar()
player2 = tk.Label(master, textvariable=player2_Var, font=('Comic Sans MS', 14), bg='LightCyan2')
player2_Var.set("Player 2")
player2.place(x=700,y=120)

In [129]:
# Function to display cards for a player with animation
def display_cards(player, card_paths, start_x, delay_offset):
    for i, path in enumerate(card_paths):
        delay = i * delay_offset  # Adjust timing for each player
        display_card(player, path, start_x + i * 75, delay, i)

# Function to load, resize, and display a card image at a specified position
def display_card(player, image_path, x, delay, i):
    def load_and_place():
        image = Image.open(image_path)
        image_resized = image.resize((60, 80), Image.ANTIALIAS)
        photo = ImageTk.PhotoImage(image_resized)
        label = tk.Label(master, image=photo)
        label.image = photo  # Keep a reference
        label.place(x=x, y=155)  # Adjust y position based on player
        
        # Create arrow buttons
        left_arrow = tk.Button(master, text="<", command=lambda: swap_cards(player, i, -1), bg="LightCyan2", font=("Arial", 10))
        right_arrow = tk.Button(master, text=">", command=lambda: swap_cards(player, i, 1), bg="LightCyan2", font=("Arial", 10))

        # Calculate the starting x-coordinate for the left button to center both buttons
        button_width = 20  # Width of each button
        button_spacing = 5  # Spacing between buttons
        total_button_width = 2 * button_width + button_spacing  # Total width of both buttons and spacing
        button_x = x + (60 - total_button_width) // 2  # Assuming the width of the card label is 60 pixels

        # Place the buttons centered under the card photo, side by side
        left_arrow.place(x=button_x, y=235)  # Adjust y position as needed
        right_arrow.place(x=button_x + button_width + button_spacing +5, y=235)  # Adjust y position as needed

        # Store both the widget and the image name
        card_info = (label, image_path)
        if player == 1:
            player1_cards.append(card_info)
        else:
            player2_cards.append(card_info)
            
    master.after(delay, load_and_place)

def save_player_cards(player_cards, player_name, button):
    card_data = {}
    for index, (label, image_path) in enumerate(player_cards, start=1):
        card_name = f"{index}.jpg"
        position = (label.winfo_x(), label.winfo_y())
        card_data[card_name] = position

    if button.cget("bg") == "grey":  # Check if the button is grey
        button.configure(bg="#90EE90")  # Change button color to green
    else:  # Button is green
        button.configure(bg="grey")  # Change button color to grey

def create_center_frame():
    center_frame = tk.Frame(master, bg="white", width=160, height=160)
    center_frame.place(x=420, y=250)
    return center_frame

def remove_cards(center_frame, card_player1_value, card_player2_value):
    global player1_cards, player2_cards

    # Determine which card should be removed based on comparison
    if card_player1_value == 1 and card_player2_value == 4:
        # Both cards are removed from the players' hands
        player1_cards.pop(0)
        player2_cards.pop(0)
    elif card_player1_value == 4 and card_player2_value == 1:
        # Both cards are removed from the players' hands
        player1_cards.pop(0)
        player2_cards.pop(0)
    elif card_player1_value < card_player2_value:
        # Player 1's card is removed from their hand
        player1_cards.pop(0)
    elif card_player1_value > card_player2_value:
        # Player 2's card is removed from their hand
        player2_cards.pop(0)
    else:
        # Cards have the same value, no removal
        player1_cards.pop(0)
        player2_cards.pop(0)

    # Update the GUI to reflect the changes
    refresh_screen()

def display_card_in_frame(frame, image_path, card_index):
    frame_width = 160  # Assuming the frame's width is 160 pixels
    card_width = 60
    spacing = 20  # Spacing between cards
    total_width = 2 * card_width + spacing
    
    # Calculate starting x-coordinate to center the cards
    start_x = (frame_width - total_width) // 2

    # Calculate the x offset for the card based on its index
    x_offset = start_x + (card_width + spacing) * card_index
    
    # The rest remains the same
    image = Image.open(image_path)
    image_resized = image.resize((card_width, 80), Image.ANTIALIAS)  # Keep the card's height as 80
    photo = ImageTk.PhotoImage(image_resized)
    label = tk.Label(frame, image=photo)
    label.image = photo  # Keep a reference to avoid garbage collection
    label.place(x=x_offset, y=40)  # Adjust y position as needed

def compare_cards():
    global player1_cards, player2_cards

    if len(player1_cards) == 0 and len(player2_cards) == 0:
        if not play_again_pressed:
            tie = tk.StringVar()
            tie2 = tk.Label(master, textvariable=tie, font=('Comic Sans MS', 14), bg='LightCyan2')
            display_winner_message("It's a tie!")

    elif len(player2_cards) == 0:
        player1_VarWIN = tk.StringVar()
        player1WIN = tk.Label(master, textvariable=player1_VarWIN, font=('Comic Sans MS', 14), bg='LightCyan2')
        display_winner_message("Player 1 won!")

    elif len(player1_cards) == 0:
        player2_VarWIN = tk.StringVar()
        player2WIN = tk.Label(master, textvariable=player2_VarWIN, font=('Comic Sans MS', 14), bg='LightCyan2')
        display_winner_message("Player 2 won!")

    # Get the names of the first cards of each player
    card_player1 = player1_cards[0][1].split("/")[-1]  # Get the card name (e.g., "1.jpg")
    card_player2 = player2_cards[0][1].split("/")[-1]  # Get the card name (e.g., "1.jpg")

    # Convert the card names to their numerical values
    card_player1_value = int(card_player1.split(".")[0])
    card_player2_value = int(card_player2.split(".")[0])

    # Create a center frame for comparison
    center_frame = create_center_frame()

    # Display cards for comparison
    display_card_in_frame(center_frame, "C:/Users/maryomaalaa/Desktop/" + card_player1, card_index=0)
    display_card_in_frame(center_frame, "C:/Users/maryomaalaa/Desktop/" + card_player2, card_index=1)

    # Wait for a short time to visualize the comparison
    master.after(2000, lambda: remove_cards(center_frame, card_player1_value, card_player2_value))
    if len(player1_cards) != 0 and len(player2_cards) != 0:
        master.after(2000, compare_cards)  # Schedule the next comparison

play_again_pressed = False

def display_initial_setup():
    # Display title and player labels
    title_var = tk.StringVar()
    title = tk.Label(master, textvariable=title_var, font=('Comic Sans MS', 20), bg='LightCyan2')
    title_var.set("Simultaneous QUATRO-UNO")
    title.place(x=295, y=27)

    player1_Var = tk.StringVar()
    player1 = tk.Label(master, textvariable=player1_Var, font=('Comic Sans MS', 14), bg='LightCyan2')
    player1_Var.set("Player 1")
    player1.place(x=200, y=120)

    player2_Var = tk.StringVar()
    player2 = tk.Label(master, textvariable=player2_Var, font=('Comic Sans MS', 14), bg='LightCyan2')
    player2_Var.set("Player 2")
    player2.place(x=700, y=120)

    # Display instructions and cards for both players
    display_instructions()
    display_cards(1, card_paths_player1, start_x_player1, delay_offset)
    display_cards(2, card_paths_player2, start_x_player2, delay_offset)

    start_button = tk.Button(master, text="Start Game", command=start_game, bg="#D3D3D3", font=("Arial", 18))
#start_button.place(x=400, y=400)

    def on_confirm_player1(button):
        save_player_cards(player1_cards, "Player 1", button)
        if confirm_button1.cget("bg") == "#90EE90" and confirm_button2.cget("bg") == "#90EE90":
            start_button.place(x=420, y=340)  # Display the "Start Game" button

    def on_confirm_player2(button):
        save_player_cards(player2_cards, "Player 2", button)
        if confirm_button1.cget("bg") == "#90EE90" and confirm_button2.cget("bg") == "#90EE90":
            start_button.place(x=420, y=340)  # Display the "Start Game" button


    # Display "Ready" buttons for both players
    confirm_button1 = tk.Button(master, text="Ready", command=lambda: on_confirm_player1(confirm_button1), bg="grey", font=("Arial", 10))
    confirm_button2 = tk.Button(master, text="Ready", command=lambda: on_confirm_player2(confirm_button2), bg="grey", font=("Arial", 10))
    delay = 3000
    master.after(delay, lambda: confirm_button1.place(x=210, y=305))
    master.after(delay, lambda: confirm_button2.place(x=720, y=305))

    update_scoreboard()

def play_again():
    global player1_cards, player2_cards, player1_score, player2_score, play_again_pressed
    play_again_pressed = True

    # Reset player cards
    player1_cards = []
    player2_cards = []

    # Clear the screen
    for widget in master.winfo_children():
        widget.destroy()

    # Redisplay the initial setup
    display_initial_setup()

    # Remove the winner announcement labels if they exist
    for widget in master.winfo_children():
        if isinstance(widget, tk.Label):
            if widget.cget("text") in ["It's a tie!", "Player 1 won!", "Player 2 won!"]:
                widget.destroy()
    # Start the game again
    start_game()
    update_scoreboard()
    

def quit_playing():
    # Close all windows and exit the game
    master.destroy()

def start_game():
    refresh_screen()
    confirm_and_display_player1()
    confirm_and_display_player2()
    compare_cards()
    update_scoreboard()


player1_score = 0
player2_score = 0
player1_score_label = None
player2_score_label = None

# Function to display winner message and update scoreboard
def display_winner_message(winner_message):
    global player1_score, player2_score, player1_score_label, player2_score_label

    winner_label = tk.Label(master, text=winner_message, font=('Comic Sans MS', 14), bg='LightCyan2')
    winner_label.place(x=440, y=200)

    # Update scores based on winner
    if winner_message == "Player 1 won!":
        player1_score += 1
    elif winner_message == "Player 2 won!":
        player2_score += 1

    # Display scoreboard
    scoreboard_label = tk.Label(master, text="Scoreboard", font=('Comic Sans MS', 14), bg='LightCyan2')
    scoreboard_label.place(x=428, y=250)

    player1_score_label = tk.Label(master, text=f"Player 1: {player1_score}", font=('Comic Sans MS', 12), bg='LightCyan2')
    player1_score_label.place(x=320, y=300)

    player2_score_label = tk.Label(master, text=f"Player 2: {player2_score}", font=('Comic Sans MS', 12), bg='LightCyan2')
    player2_score_label.place(x=550, y=300)

    # Display "Play Again" and "Quit Playing" buttons
    play_again_button = tk.Button(master, text="Play Again", command=play_again, bg="#D3D3D3", font=("Arial", 14))
    quit_playing_button = tk.Button(master, text="Quit Playing", command=quit_playing, bg="#D3D3D3", font=("Arial", 14))
    play_again_button.place(x=350, y=350)
    quit_playing_button.place(x=500, y=350)

# Function to update the scoreboard
def update_scoreboard():
    global player1_score, player2_score
    player1_score_label.config(text=f"Player 1: {player1_score}")
    player2_score_label.config(text=f"Player 2: {player2_score}")
    
def refresh_screen():
    # Clear the screen
    for widget in master.winfo_children():
        widget.destroy()
    
    # Display the "Game Board" text
    game_board_label = tk.Label(master, text="Game Board", font=('Comic Sans MS', 20), bg='LightCyan2')
    game_board_label.place(x=400, y=50)

    # Create a frame for Player 1's ordered cards
    frame_player1 = tk.Frame(master, bg="white")
    frame_player1.place(x=100, y=150)
    player1_VarNEW = tk.StringVar()
    player1NEW = tk.Label(master, textvariable=player1_VarNEW, font=('Comic Sans MS', 14), bg='LightCyan2')
    player1_VarNEW.set("Player 1")
    player1NEW.place(x=210,y=100)
    display_ordered_cards(frame_player1, 1)

    # Create a frame for Player 2's ordered cards
    frame_player2 = tk.Frame(master, bg="white")
    frame_player2.place(x=600, y=150)
    player2_VarNEW = tk.StringVar()
    player2NEW = tk.Label(master, textvariable=player2_VarNEW, font=('Comic Sans MS', 14), bg='LightCyan2')
    player2_VarNEW.set("Player 2")
    player2NEW.place(x=710,y=100)
    display_ordered_cards(frame_player2, 2)

    

# Create the "Start the Game" button (but don't place it yet)
start_button = tk.Button(master, text="Start Game", command=start_game, bg="#D3D3D3", font=("Arial", 18))
#start_button.place(x=400, y=400)

def on_confirm_player1(button):
    save_player_cards(player1_cards, "Player 1", button)
    if confirm_button1.cget("bg") == "#90EE90" and confirm_button2.cget("bg") == "#90EE90":
        start_button.place(x=420, y=340)  # Display the "Start Game" button

def on_confirm_player2(button):
    save_player_cards(player2_cards, "Player 2", button)
    if confirm_button1.cget("bg") == "#90EE90" and confirm_button2.cget("bg") == "#90EE90":
        start_button.place(x=420, y=340)  # Display the "Start Game" button

# Function to confirm and display ordered cards for Player 1
def confirm_and_display_player1():
    ordered_card_paths_player1 = [card_info[1] for card_info in player1_cards]

# Function to confirm and display ordered cards for Player 2
def confirm_and_display_player2():
    ordered_card_paths_player2 = [card_info[1] for card_info in player2_cards]

confirm_button1 = tk.Button(master, text="Ready", command=lambda: on_confirm_player1(confirm_button1), bg="grey", font=("Arial", 10))
confirm_button2 = tk.Button(master, text="Ready", command=lambda: on_confirm_player2(confirm_button2), bg="grey", font=("Arial", 10))

delay = 5000  # 5000 ms = 5 seconds, adjust based on your actual delay
master.after(delay, lambda: confirm_button1.place(x=210, y=305))
master.after(delay, lambda: confirm_button2.place(x=720, y=305))

# Function to swap cards
def swap_cards(player, card_index, direction):
    if player == 1:
        card_list = player1_cards
    else:
        card_list = player2_cards
    
    if 0 <= card_index + direction < len(card_list):
        card_list[card_index], card_list[card_index + direction] = card_list[card_index + direction], card_list[card_index]
        update_card_positions(player)

# Function to update card positions after swapping
def update_card_positions(player):
    if player == 1:
        card_list = player1_cards
        start_x = start_x_player1
    else:
        card_list = player2_cards
        start_x = start_x_player2
    
    for i, (label, _) in enumerate(card_list):  # Unpack each tuple to get the label
        new_x = start_x + i * 75
        label.place(x=new_x, y=155)  # Use label to place

# Function to display ordered cards for each player within a given frame
def display_ordered_cards(frame, player):
    ordered_card_paths = [card_info[1] for card_info in player1_cards] if player == 1 else [card_info[1] for card_info in player2_cards]

    # Display ordered cards
    for i, path in enumerate(ordered_card_paths):
        image = Image.open(path)
        image_resized = image.resize((60, 80), Image.ANTIALIAS)
        photo = ImageTk.PhotoImage(image_resized)
        label = tk.Label(frame, image=photo)
        label.image = photo
        label.grid(row=1, column=i, padx=5)

# Paths to your card images
card_paths_player1 = [
    "C:/Users/maryomaalaa/Desktop/1.jpg",
    "C:/Users/maryomaalaa/Desktop/2.jpg",
    "C:/Users/maryomaalaa/Desktop/3.png",
    "C:/Users/maryomaalaa/Desktop/4.jpg"
]

card_paths_player2 = [
    "C:/Users/maryomaalaa/Desktop/1.jpg",
    "C:/Users/maryomaalaa/Desktop/2.jpg",
    "C:/Users/maryomaalaa/Desktop/3.png",
    "C:/Users/maryomaalaa/Desktop/4.jpg"
]

# Start positions for player 1 and player 2
start_x_player1 = 100
start_x_player2 = 600

# Offset for timing of card animations for player 1 and player 2
delay_offset = 600  # Adjust as needed

# Delay before starting card animation
initial_delay = 2000  # 2 seconds delay before showing the first card

player1_cards = []  # List to hold player 1's cards (labels)
player2_cards = []  # List to hold player 2's cards (labels)

# Display cards for player 1 and player 2 with animation after initial delay
master.after(initial_delay, lambda: display_cards(1, card_paths_player1, start_x_player1, delay_offset))
master.after(initial_delay, lambda: display_cards(2, card_paths_player2, start_x_player2, delay_offset))

# Function to display instructions after all card animations
def display_instructions():
    player1_Var1 = tk.StringVar()
    player11 = tk.Label(master, textvariable=player1_Var1, font=('Comic Sans MS', 14), bg='LightCyan2')
    player1_Var1.set("Order your cards into a pile")
    player11.place(x=120, y=270)

    player2_Var2 = tk.StringVar()
    player22 = tk.Label(master, textvariable=player2_Var2, font=('Comic Sans MS', 14), bg='LightCyan2')
    player2_Var2.set("Order your cards into a pile")
    player22.place(x=620, y=270)

# Calculate when all animations are done (assuming 4 cards per player and known delay_offset)
total_animation_time = initial_delay + max(len(card_paths_player1), len(card_paths_player2)) * delay_offset

# Schedule the display of instructions
master.after(total_animation_time, display_instructions)

'after#841'

In [130]:
master.mainloop()

C:\Users\maryomaalaa\AppData\Local\Temp\ipykernel_35072\2924892219.py:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image_resized = image.resize((60, 80), Image.ANTIALIAS)
C:\Users\maryomaalaa\AppData\Local\Temp\ipykernel_35072\2924892219.py:355: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image_resized = image.resize((60, 80), Image.ANTIALIAS)
C:\Users\maryomaalaa\AppData\Local\Temp\ipykernel_35072\2924892219.py:97: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image_resized = image.resize((card_width, 80), Image.ANTIALIAS)  # Keep the card's height as 80
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n

In [131]:
# Define the payoff matrix
payoff_matrix = [
    [0, 1, -1, 0],  # Player 1 plays 1
    [-1, 0, 1, -1], # Player 1 plays 2
    [1, -1, 0, 1],  # Player 1 plays 3
    [0, 1, -1, 0]  # Player 1 plays 4
]

# Convert the payoff matrix to a DataFrame
df = pd.DataFrame(payoff_matrix, columns=[1, 2, 3, 4], index=[1, 2, 3, 4])

# Export the DataFrame to an Excel file
df.to_excel("payoff_matrix.xlsx")

# You can specify the path where you want to save the Excel file
# For example, if you want to save it in the current directory:
# df.to_excel("payoff_matrix.xlsx", index_label="Action")

print("Payoff matrix exported to 'payoff_matrix.xlsx'")

Payoff matrix exported to 'payoff_matrix.xlsx'
